In [7]:
import numpy as np
import pandas as pd 
import datetime as dt
from datetime import timedelta 
import random

In [8]:
class Driver: 
    

    
    def __init__(self, time, location, total_fare, total_cost, is_idle ):
        self.time = time 
        self.location = location 
        self.total_fare = total_fare
        self.total_cost  = total_cost
        self.is_idle = is_idle
    
    def set_time(self, new_time):
        self.time = new_time
    
    def get_time(self):
        return self.__time 
    
    def set_location(self, new_location):
        self.location = new_location
        
    def get_location(self):
        return self.__location 
   
    def set_total_fare(self, new_fare):
        self.total_fare = new_fare
    
    def get_total_fare(self):
        return self.total_fare 
    
    def set_is_idle(self, is_idle):
        self.is_idle = is_idle 
        
    def set_total_cost(self, new_cost):
        self.total_cost = new_cost
    
    def get_total_cost(self):
        return self.total_cost 
    
    


In [9]:
class Env: 
    
    
    arrivals = pd.read_csv('../data/arrival_rates.csv')[['taxi_zone', 'time_stamp','arr_weekday','arr_weekend']]
    dropoff_counts = pd.read_csv('../data/pickup_dropoff_summary.csv')
    fare_df = pd.read_csv('../data/fare_amount_scr_dst_t.csv')
    dist_df = pd.read_csv('../data/trip_distance_scr_dst.csv')
    trip_time_df = pd.read_csv('../data/trip_time_scr_dst_t.csv')
    prob_df = pd.read_csv('../data/prob_match_combined_new.csv')
    trans_matrix = pd.read_csv('../data/dropoff_after_pickup.csv') # fix this to shift based and a
     # 0 weekend , A night shift 
    
    #time in 1..48,  isWeekday is a boolean
    def __init__(self,  isWeekday):   

        self.isWeekday = isWeekday 
        
    
    def is_matched(self, matching_prob):
        n = random.random()

        if n <= matching_prob:
            return True 
        else: 
            False 
    
    def get_destination(self, zone):
        prob = self.trans_matrix.iloc[zone-1].values[1:]
        n = random.random()

        count = 0
        for i in range(len(prob)):
            count = count + prob[i]
            if n<count:
                return i+1

    def get_adjacent_zone( self, taxi_zone ):
        df = pd.read_csv('../data/adjacent_zone.csv')
        adj_zone_1 = df.loc[df["zone1"]==taxi_zone, "zone2"].values.tolist()
        adj_zone_2 = df.loc[df["zone2"]==taxi_zone, "zone1"].values.tolist()
        adj_zone_list = list(filter(None, adj_zone_1+adj_zone_2))
        return(adj_zone_list)
    
    def get_demand(self, zone, time):
        #row = self.arrivals.loc[ ( self.arrivals['taxi_zone'] == zone ) & (self.arrivals['time_stamp'] == time ) ]
        row = self.prob_df.loc[ ( self.arrivals['taxi_zone'] == zone ) & (self.arrivals['time_stamp'] == time ) ]
       
        if self.isWeekday:
            return row['arr_weekday'].values[0]
        else:
            return row['arr_weekend'].values[0]
    
#  def get_near_arrival():
    
    
    '''TODO： 
        fix policy fucntion - add randomness'''
    
    def reposition_to( self, current_zone, time ):
        #adj = self.get_adjacent_zone( current_zone )
        #adj.append( current_zone )
        #demand_list = [self.get_demand(zone, time) for zone in adj]
        #print(demand_list)
        # find the index of the max value of demand_lisy, and find which zone it is
        #return adj[demand_list.index( max(demand_list))]  
    
         
        adj = self.get_adjacent_zone( current_zone )
        adj.insert(0,current_zone)
        # find the index of the max value of near_prob, and find which zone it is 
        demand_list = [self.get_demand(zone, time) for zone in adj]
        a_arr  = np.array(demand_list)
        print("a_arr: ", a_arr)
        #near_prob = list(np.exp((a_arr-a_arr[0])*10)/np.sum(np.exp((a_arr-a_arr[0])*10))) 
        near_prob = list(np.exp((a_arr-a_arr[0]))/np.sum(np.exp((a_arr-a_arr[0]))))
        print("near prob :", near_prob)  
        
        r = random.random()
            
        
        repo_prob =  near_prob.index( max(near_prob))   
        repo = adj[near_prob.index( max(near_prob))]
        if r <= repo_prob:
            is_repositioned = True
        else :
            is_repositioned = False
        
        print("r , repo_prob , repo : " , r, repo_prob, repo)
        
        return(repo,is_repositioned )
        #return (adj[near_prob.index( max(near_prob))],  is_repositioned)
    
        #SOFTMAX 
        #NEAR ARRIVAL 
        # return the highest prob of near arrival 
        
        
        ### paper's calculation of nearest prob
        # zone 1 = 10 , zone2 = 10.1 , zone 3 = 5
        # p(zone 1) = 10/25.5 = 0.392
        # p(zone 2) = 10.1/25.5 = 0.396
        # p(zone 3) = 5/25.5 = 0.196 
        
        
        # softmax suggestion way 
        # zone 1 = 10 , zone2 = 10.1 , zone 3 = 5
        # B  = 10 
        #exp(B * (rho1 - rho1) - e) / exp(B * (rho1 - rho1)) + exp(B * (rho2 - rho1)) + exp(B * (rho3 - rho1) - e) = 0.27
        #exp(B * (rho2 - rho1) - e) / exp(10) + exp(10.2) + exp(5)  = 0.73
        #exp(B * (rho3 - rho1) - e) / exp(10) + exp(10.2) + exp(5)  = 0.0
        
        
     
    def wait_time_before_pickup(self, matching_prob):
        if ( matching_prob >= 0.9 ):
            return random.randint(1,4)
        elif ( matching_prob >= 0.7 ):
            return random.randint(5,10)
        elif ( matching_prob >= 0.5 ):
            return random.randint(11,15)
        elif ( matching_prob >= 0.3 ):
            return random.randint(16,20)
        else:
            return random.randint(21,29)
        
    def wait_time_before_reposition(self):
        return random.randint(1,10)
    
    # can look into histortical data combined with random number 

    
    
    '''TODO: 
        fix the three functions below '''
    def get_trip_duration(self, start_zone, end_zone):    
        dur = self.trip_time_df.loc[(
            (self.trip_time_df["pickup_taxizone_id"] == start_zone )  & 
            (self.trip_time_df["dropoff_taxizone_id"] == end_zone)), "mean"].describe().mean()  
        
        return dur
        
    def get_fare(self, start_zone, end_zone): 
        fare = self.fare_df.loc[((self.fare_df["pickup_taxizone_id"] == start_zone) & 
                                 (self.fare_df["dropoff_taxizone_id"] == end_zone)), "mean"].describe().mean()
        return fare
    
    def get_trip_distance(self, start_zone, end_zone):
        dist = self.dist_df.loc[((self.dist_df["pickup_taxizone_id"] == start_zone) &
                                 (self. dist_df["dropoff_taxizone_id"] == end_zone)), "mean"].describe().mean()
        
        return dist 
    
    def get_fuel_cost(self, start_zone, end_zone): 
        
        cost = self.dist_df.loc[((self.dist_df["pickup_taxizone_id"] == start_zone) &
                                 (self. dist_df["dropoff_taxizone_id"] == end_zone)), "mean"].describe().mean()*0.124
        return cost 
    
    def get_matching_prob(self, zone, time):
        if self.isWeekday:
            prob = self.prob_df.loc[(self.prob_df['taxi_zone'] == zone) & 
                                    (self.prob_df['time_stamp'] == time)].mod_prob_match_weekday.values[0]
        else: 
            prob = self.prob_df.loc[(self.prob_df['taxi_zone'] == zone) & 
                                    (self.prob_df['time_stamp'] == time)].mod_prob_match_weekend.values[0]
            
        return prob 
    
    
    time_bins = [dt.time(0,0),dt.time(0,30), dt.time(1,0), dt.time(1,30), dt.time(2,0), 
        dt.time(2,30), dt.time(3,0), dt.time(3,30), dt.time(4,0),
        dt.time(4,30), dt.time(5,0), dt.time(5,30), dt.time(6,0),
        dt.time(6,30), dt.time(7,0), dt.time(7,30), dt.time(8,0),
        dt.time(8,30), dt.time(9,0), dt.time(9,30), dt.time(10,0),
        dt.time(10,30), dt.time(11,0), dt.time(11,30), dt.time(12,0),
        dt.time(12,30), dt.time(13,0), dt.time(13,30), dt.time(14,0),
        dt.time(14,30), dt.time(15,0), dt.time(15,30), dt.time(16,0), 
        dt.time(16,30), dt.time(17,0), dt.time(17,30), dt.time(18,0),
        dt.time(18,30), dt.time(19,0), dt.time(19,30), dt.time(20,0),
        dt.time(20,30), dt.time(21,0), dt.time(21,30), dt.time(22,0),
        dt.time(22,30), dt.time(23,0), dt.time(23,30), dt.time(23,59,59)] 
    
    
    def get_time_window(self, datetime):
        for i in range(0,48):
            if (datetime.time() >= self.time_bins[i]) & (datetime.time() < self.time_bins[i+1]): 
                return i+1 


In [10]:
# one round of simulation 

init_time = dt.datetime(2013, 6, 3, 5, 0, 0) 
init_zone = 66

#Initialize enviorment and driver 
env = Env( isWeekday = True ) 
driver = Driver( time = init_time, location = init_zone, total_fare = 0, total_cost = 0, is_idle = True) 

# print(driver.location)
# print(driver.time)
# print(driver.total_fare)
# print(driver.total_cost)
# print(driver.is_idle)


while driver.time < init_time + timedelta(seconds=0, minutes=0, hours = 11):
    #get prob that driver will be matched to a passenger 
    matching_prob = env.get_matching_prob( zone = driver.location, time = env.get_time_window(driver.time) )
    #print(matching_prob)
    start_zone = driver.location
    print('current time is ', driver.time, 'and driver is at zone ', driver.location)
    if env.is_matched(matching_prob):

        # time it takes before getting matched 
        wait_time = env.wait_time_before_pickup(matching_prob)
        print('driver waited for ', wait_time, ' min before he picks up')

        # set driver time to new time 
        driver.set_time( driver.time + timedelta(seconds=0, minutes= wait_time, hours=0 ) )
        print('driver gets a pick up and is now at time ', driver.time)    


        #driver is now busy 
        driver.set_is_idle(False)


        # find the destination of the trip and set destination 
        dest = env.get_destination(driver.location)
        driver.set_location(dest)
        print ('driver is going to zone', dest, 'and driver arrived at zone', driver.location)    

         # find the trip duration and update new time 
        duration = env.get_trip_duration(start_zone = start_zone, end_zone = dest)
        print ('trip duration ', duration)
        driver.set_time( driver.time + timedelta(seconds=0, minutes= duration, hours=0 ) )
        print ('driver drops off passenger at time ', driver.time)


        #find trip fare and update fare 
        fare = env.get_fare( start_zone, dest)
        driver.set_total_fare(driver.total_fare + fare)
        print('trip fare from ', start_zone, 'to ', driver.location,   ' is',  fare)
        print('driver total fare earned is ', driver.total_fare )

        #frin trip cost and update cost 
        cost = env.get_fuel_cost(start_zone, dest)
        driver.set_total_cost(driver.total_cost + cost)
        print('fuel cost from ', start_zone, 'to ', driver.location,   ' is', cost  )

        print('************************trip is completed.************************** ')


    else: 
        #time it takes before reposition 
        print('driver does not get a pick up at the current zone: zone ', driver.location )
        
        wait_time = env.wait_time_before_reposition()
        driver.set_time( driver.time + timedelta(seconds=0, minutes= wait_time, hours=0 ) )
        print('driver now decides to check reposition info after ', wait_time, ' min of crusing and current time is ', driver.time)
        '''
            TODO:
            add cost - assume 10 miles a hour '''
        
        repo,repo_status = env.reposition_to(driver.location, env.get_time_window(driver.time) ) #fix this to distribution 
        
        while(repo_status == False ):
            wait_time = env.wait_time_before_reposition()
            driver.set_time( driver.time + timedelta(seconds=0, minutes= wait_time, hours=0 ) )
            print('driver now decides to check reposition info again after ', wait_time, ' min of crusing and current time is ', driver.time)
            repo,repo_status = env.reposition_to(driver.location, env.get_time_window(driver.time) ) #fix this to distribution 
            '''
            TODO:
            add cost - assume 10 miles a hour '''
                                                                       
                
        driver.set_location(repo)
        print('driver is repostioning to zone ', repo, 'and arrived at zone ', driver.location)

        # find the trip duration and update new time 
        duration = env.get_trip_duration(start_zone = start_zone, end_zone = repo)
        print ('reposition trip duration ', duration , 'min')
        driver.set_time( driver.time + timedelta(seconds=0, minutes= duration, hours=0 ) )
        print ('After repositioning, it is now ', driver.time)

        #frin trip cost and update cost 
        cost = env.get_fuel_cost(start_zone, repo)
        driver.set_total_cost(driver.total_cost + cost)
        print('fuel cost from ', start_zone, 'to ', driver.location,   ' is', cost  )


        print('************************reposition is completed.************************** ')


        
print('total fare earned from this shift is ', driver.total_fare )
print('total fuel cost is ', driver.total_cost)
print('profit is ', driver.total_fare - driver.total_cost)

current time is  2013-06-03 05:00:00 and driver is at zone  66
driver does not get a pick up at the current zone: zone  66
driver now decides to check reposition info after  6  min of crusing and current time is  2013-06-03 05:06:00
a_arr:  [1.25173828 3.94211426 5.04131653 0.         3.70111206]
near prob : [0.013919423850973337, 0.20513355208272754, 0.6157638385844674, 0.003981055490611073, 0.1612021299912208]
r , repo_prob , repo :  0.14437718099290942 2 33
driver is repostioning to zone  33 and arrived at zone  33
reposition trip duration  11.136815077546046 min
After repositioning, it is now  2013-06-03 05:17:08.208905
fuel cost from  66 to  33  is 0.1436826258205689
************************reposition is completed.************************** 
current time is  2013-06-03 05:17:08.208905 and driver is at zone  33
driver waited for  1  min before he picks up
driver gets a pick up and is now at time  2013-06-03 05:18:08.208905
driver is going to zone 79 and driver arrived at zone 79
tr

current time is  2013-06-03 10:23:23.799217 and driver is at zone  90
driver waited for  2  min before he picks up
driver gets a pick up and is now at time  2013-06-03 10:25:23.799217
driver is going to zone 170 and driver arrived at zone 170
trip duration  13.43251429638645
driver drops off passenger at time  2013-06-03 10:38:49.750075
trip fare from  90 to  170  is 12.451540301631395
driver total fare earned is  225.3118126321483
fuel cost from  90 to  170  is 0.17651757162744447
************************trip is completed.************************** 
current time is  2013-06-03 10:38:49.750075 and driver is at zone  170
driver waited for  1  min before he picks up
driver gets a pick up and is now at time  2013-06-03 10:39:49.750075
driver is going to zone 161 and driver arrived at zone 161
trip duration  11.818420917665819
driver drops off passenger at time  2013-06-03 10:51:38.855330
trip fare from  170 to  161  is 11.272384843940072
driver total fare earned is  236.58419747608838
fue

In [11]:
'''TODO: 
Monte Carlo Simulation'''

'TODO: \nMonte Carlo Simulation'

In [12]:
# # one round of simulation 

# init_time = dt.datetime(2013, 6, 3, 5, 0, 0) 
# init_zone = 66

# #Initialize enviorment and driver 
# env = Env( isWeekday = True ) 
# driver = Driver( time = init_time, location = init_zone, total_fare = 0, total_cost = 0, is_idle = True) 

# # print(driver.location)
# # print(driver.time)
# # print(driver.total_fare)
# # print(driver.total_cost)
# # print(driver.is_idle)

# #get prob that driver will be matched to a passenger 
# matching_prob = env.get_matching_prob( zone = driver.location, time = env.get_time_window(driver.time) )
# #print(matching_prob)
# start_zone = driver.location
# print('current time is ', driver.time, 'and driver is at zone ', driver.location)
# if env.is_matched(matching_prob):
    
#     # time it takes before getting matched 
#     wait_time = env.wait_time_before_pickup(matching_prob)
#     print('driver waited for ', wait_time, ' min before he picks up')
    
#     # set driver time to new time 
#     driver.set_time( driver.time + timedelta(seconds=0, minutes= wait_time, hours=0 ) )
#     print('driver gets a pick up and is now at time ', driver.time)    
    
    
#     #driver is now busy 
#     driver.set_is_idle(False)
    
                        
#     # find the destination of the trip and set destination 
#     dest = env.get_destination(driver.location)
#     driver.set_location(dest)
#     print ('driver is going to zone', dest, 'and driver arrived at zone', driver.location)    
    
#      # find the trip duration and update new time 
#     duration = env.get_trip_duration(start_zone = start_zone, end_zone = dest)
#     print ('trip duration ', duration)
#     driver.set_time( driver.time + timedelta(seconds=0, minutes= duration, hours=0 ) )
#     print ('driver drops off passenger at time ', driver.time)

    
#     #find trip fare and update fare 
#     fare = env.get_fare( start_zone, dest)
#     driver.set_total_fare(driver.total_fare + fare)
#     print('trip fare from ', start_zone, 'to ', driver.location,   ' is',  fare)
#     print('driver total fare earned is ', driver.total_fare )
    
#     #frin trip cost and update cost 
#     cost = env.get_fuel_cost(start_zone, dest)
#     driver.set_total_cost(driver.total_cost + cost)
#     print('fuel cost from ', start_zone, 'to ', driver.location,   ' is', cost  )
    
#     print('************************trip is completed.************************** ')
    

# else: 
#     #time it takes before reposition 
#     print('driver does not get a pick up at the current zone: zone ', driver.location )
#     wait_time = env.wait_time_before_reposition()
#     driver.set_time( driver.time + timedelta(seconds=0, minutes= wait_time, hours=0 ) )
#     print('driver now decides to reposition after ', wait_time, ' min of crusing and current time is ', driver.time)
    
#     #find where to reposition to and update location 
#     repo = env.reposition_to(driver.location, env.get_time_window(driver.time) ) #fix this to distribution 
#     driver.set_location(repo)
#     print('driver is repostioning to zone ', repo, 'and arrived at zone ', driver.location)
    
#     # find the trip duration and update new time 
#     duration = env.get_trip_duration(start_zone = start_zone, end_zone = repo)
#     print ('reposition trip duration ', duration , 'min')
#     driver.set_time( driver.time + timedelta(seconds=0, minutes= duration, hours=0 ) )
#     print ('After repositioning, it is now ', driver.time)
                    
#     #frin trip cost and update cost 
#     cost = env.get_fuel_cost(start_zone, repo)
#     driver.set_total_cost(driver.total_cost + cost)
#     print('fuel cost from ', start_zone, 'to ', driver.location,   ' is', cost  )
                    
                    
#     print('************************reposition is completed.************************** ')

